In [1]:
import xarray as xr
import numpy as np
import glob 
import xgcm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap, cm, shiftgrid
import dask.array as da

from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=3, threads_per_worker=20)
client = Client(cluster)

/HOME/users/fcampos/miniconda3/lib/python3.9/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42480 instead
  warnings.warn(
2022-12-29 04:58:48,479 - distributed.diskutils - INFO - Found stale lock file and directory '/HOME/users/fcampos/scripts/dask-worker-space/worker-rw7hvxbb', purging


In [2]:
uname = "/HOME/users/fcampos/outputs/global/U/"
vname = "/HOME/users/fcampos/outputs/global/V/"
gridname = "/HOME/users/fcampos/outputs/global/grid/"
nx, ny = 8640, 6480

XC = np.memmap(gridname+'XC.data',dtype='>f4',shape=(ny,nx),mode='r+')
YC = np.memmap(gridname+'YC.data',dtype='>f4',shape=(ny,nx),mode='r+')
XG = np.memmap(gridname+'XG.data',dtype='>f4',shape=(ny,nx),mode='r+')
YG = np.memmap(gridname+'YG.data',dtype='>f4',shape=(ny,nx),mode='r+')
dxC = np.memmap(gridname+'dxC.data',dtype='>f4',shape=(ny,nx),mode='r+')
dyC = np.memmap(gridname+'dyC.data',dtype='>f4',shape=(ny,nx),mode='r+')
dxG = np.memmap(gridname+'dxG.data',dtype='>f4',shape=(ny,nx),mode='r+')
dyG = np.memmap(gridname+'dyG.data',dtype='>f4',shape=(ny,nx),mode='r+')
drF = np.memmap(gridname+'drF.data',dtype='>f4',shape=(ny,nx),mode='r+')
rAZ = np.memmap(gridname+'rAz.data',dtype='>f4',shape=(ny,nx),mode='r+')
rA = np.memmap(gridname+'rA.data',dtype='>f4',shape=(ny,nx),mode='r+')
hfacs=np.memmap(gridname+'hFacS.data',dtype='>f4',shape=(ny,nx),mode='r+')
hfacw=np.memmap(gridname+'hFacW.data',dtype='>f4',shape=(ny,nx),mode='r+')
hfacc=np.memmap(gridname+'hFacC.data',dtype='>f4',shape=(ny,nx),mode='r+')

coords={
    "i": (["i"], np.arange(nx), {"axis": "X"}, ),
    "j": (["j"], np.arange(ny), {"axis": "Y"}, ), 
    "i_g": (["i_g"], np.arange(nx), {"axis": "X", "c_grid_axis_shift": -0.5}, ),
    "j_g": (["j_g"], np.arange(ny), {"axis": "Y", "c_grid_axis_shift": -0.5}, ),
    "XC": (["j", "i"], XC, ),
    "XG": (["j_g", "i_g"], XG, ),
    "YC": (["j", "i"], YC, ),
    "YG": (["j_g", "i_g"], YG, ),
    "dxC": (["j", "i_g"], dxC, ),
    "dyC": (["j_g", "i"], dyC, ),    
    "dxG": (["j_g", "i"], dxG, ),
    "dyG": (["j", "i_g"], dyG, ),    
    "rAz": (["j_g", "i_g"], rAZ, ),
    "rA": (["j", "i"], rA, ),    
    "hFacS": (["j_g", "i"], hfacs, ),
    "hFacW": (["j", "i_g"], hfacw, ),
    "hFacC": (["j", "i"], hfacc, ),
    }

In [3]:
uf = np.sort(glob.glob(uname+'jfm/U_8640x6480_201201*'))
vf = np.sort(glob.glob(vname+'jfm/V_8640x6480_201201*'))

nt=len(uf)

u = da.zeros((nt, ny, nx), chunks=(24, 2160, 2160))
v = da.zeros((nt, ny, nx), chunks=(24, 2160, 2160))

for i in range(nt):
    u[i,:,:] = np.squeeze(np.memmap(uf[i],dtype='>f4',shape=(ny,nx),mode='r+'))
    v[i,:,:] = np.squeeze(np.memmap(vf[i],dtype='>f4',shape=(ny,nx),mode='r+'))

data_vars = dict(    
    u = (["time","j","i_g"], u, {"units":"m/s"}),
    v = (["time","j_g","i"], v, {"units":"m/s"})
)

del u, v, uf, vf

ds = xr.Dataset(data_vars=data_vars, 
                coords=coords, 
                )

In [4]:
grid = xgcm.Grid(ds, periodic=["X", "Y"])
ke = 0.5*(grid.interp((ds.u*ds.hFacW)**2, 'X') + grid.interp((ds.v*ds.hFacS)**2, 'Y'))
ke_u = grid.interp(ke, "X")
ke_v = grid.interp(ke, "Y")
advx = ke_u * ds.u * ds.dyG * ds.hFacW 
advy = ke_v * ds.v * ds.dxG * ds.hFacS
volume = ds.hFacC * ds.rA
KE_adv = np.squeeze((grid.diff(advx, "X") + grid.diff(advy, "Y")) /  volume)

In [ ]:
(KE_adv**2).sum(axis=0).to_netcdf("/HOME/users/fcampos/outputs/01.nc")

2022-12-29 05:56:24,628 - distributed.nanny - WARNING - Restarting worker


In [ ]:
#(KE_adv.mean(axis=0)*86400).plot(vmin=-5e-1, vmax=5e-1, cmap="PiYG", size=8)
#plt.tight_layout()